In [71]:
import json
import numpy as np
import pandas as pd
pd.options.display.max_columns = None
import matplotlib.pyplot as plt

In [72]:
def check_avg(df_, season, team, stats = ['AST']):
    avg_stats = ['AVG_'+stat for stat in stats]
    df = df_.loc[(df_['SEASON_ID'] == season)& (df_['TEAM_ABBREVIATION'] == team)].sort_values('GAME_DATE')
    avg = df[stats].expanding().mean().shift(1)
    diff = df[avg_stats].values - avg[stats].values
    return(np.max(diff[1:]), np.min(diff[1:]))

In [73]:
STATS = ['PTS', 'FG2M', 'FG2A', 'FG2_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'DREB', 'OREB', 'AST', 'STL', 'BLK', 'TOV', 'PF']
AVG_STATS = ['AVG_'+stat for stat in STATS]
OPP_AVG_STATS = ['OPP_'+stat for stat in AVG_STATS]

In [74]:
df = pd.read_csv('dat/1983-2017.csv', index_col=0).sort_values(['SEASON_ID', 'TEAM_ID', 'GAME_DATE'])
df = df.set_index('GAME_ID')
df['FG3A'] = df['FG3A'].fillna(0)
df.loc[df['FG3A']==0, ['FG3M', 'FG3_PCT']] = 0
df['FG2M'] = df['FGM'] - df['FG3M']
df['FG2A'] = df['FGA'] - df['FG3A']
df['FG2_PCT'] = df['FG2M'] / df['FG2A']
df['PLUS_MINUS'] = df['PTS'] - df['PTS_A']
df = df.reset_index()

In [75]:
for season in df.loc[df[STATS].isna().any(axis=1)]['SEASON_ID'].values:
    df = df.loc[df['SEASON_ID']!=season]

In [76]:
df[STATS].isna().any(axis=0)

PTS        False
FG2M       False
FG2A       False
FG2_PCT    False
FG3M       False
FG3A       False
FG3_PCT    False
FTM        False
FTA        False
FT_PCT     False
DREB       False
OREB       False
AST        False
STL        False
BLK        False
TOV        False
PF         False
dtype: bool

In [77]:
df.sample(10)

,GAME_ID,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,PTS_A,FG2M,FG2A,FG2_PCT,WINS,PTS_AVG,PTS_A_AVG,TOTAL_GAMES,TOTAL_WINS,GP,PYE,PYE_WINS,PYE_WINS_ERROR,PYE_WINS_ERROR_ABS,PYE_PROJ,PYE_PROJ_ERROR,PYE_PROJ_ERROR_ABS,year
49609,20601227,22006,1610612747,LAL,Los Angeles Lakers,2007-04-18,LAL @ SAC,W,240,117,45,79,0.570,2,14.0,0.143,25,31,0.806,11.0,36.0,47.0,19,6.0,6.0,14.0,26.0,11,106,43,65.0,0.661538,42.0,103.341463,103.414634,82,42.0,82,0.497539,40.798170,1.201830,1.201830,42.000000,0.000000,0.000000,2006-07
23726,29400392,21994,1610612752,NYK,New York Knicks,1995-01-06,NYK @ CLE,W,240,103,38,75,0.507,11,18.0,0.611,16,19,0.842,10.0,25.0,35.0,25,9.0,6.0,13.0,23.0,10,93,27,57.0,0.473684,17.0,98.241379,97.758621,82,55.0,29,0.517124,14.996592,2.003408,2.003408,44.407565,10.592435,10.592435,1994-95
17899,29100310,21991,1610612764,WAS,Washington Bullets,1991-12-18,WAS @ SAN,W,240,96,36,87,0.414,6,14.0,0.429,18,21,0.857,11.0,28.0,39.0,18,13.0,3.0,17.0,27.0,3,93,30,73.0,0.410959,7.0,100.083333,105.375000,82,25.0,24,0.328126,7.875017,-0.875017,0.875017,26.031292,-1.031292,1.031292,1991-92
64772,21200941,22012,1610612758,SAC,Sacramento Kings,2013-03-10,SAC vs. MIL,L,240,113,44,94,0.468,6,22.0,0.273,19,25,0.760,10.0,33.0,43.0,23,7.0,3.0,10.0,25.0,-2,115,38,72.0,0.527778,22.0,99.555556,105.523810,82,30.0,63,0.307922,19.399102,2.600898,2.600898,27.850523,2.149477,2.149477,2012-13
69763,21400802,22014,1610612759,SAS,San Antonio Spurs,2015-02-19,SAS @ LAC,L,241,115,43,84,0.512,10,28.0,0.357,19,23,0.826,5.0,33.0,38.0,34,6.0,2.0,16.0,32.0,-4,119,33,56.0,0.589286,34.0,100.925926,97.148148,82,55.0,54,0.629638,34.000434,-0.000434,0.000434,51.629855,3.370145,3.370145,2014-15
27629,29601131,21996,1610612743,DEN,Denver Nuggets,1997-04-12,DEN @ LAC,L,239,94,34,76,0.447,7,17.0,0.412,19,25,0.760,8.0,27.0,35.0,13,4.0,2.0,19.0,16.0,-22,116,27,59.0,0.457627,20.0,98.115385,104.217949,82,21.0,78,0.301675,23.530688,-3.530688,3.530688,21.206702,-0.206702,0.206702,1996-97
32066,29900069,21999,1610612739,CLE,Cleveland Cavaliers,1999-11-11,CLE vs. SEA,L,240,103,42,103,0.408,7,16.0,0.438,12,17,0.706,19.0,35.0,54.0,26,5.0,5.0,12.0,24.0,-6,109,35,87.0,0.402299,3.0,98.000000,96.600000,82,32.0,5,0.549870,2.749352,0.250648,0.250648,45.340026,-13.340026,13.340026,1999-00
20611,29300405,21993,1610612741,CHI,Chicago Bulls,1994-01-07,CHI @ WAS,W,240,99,43,93,0.462,0,4.0,0.000,13,21,0.619,16.0,31.0,47.0,22,9.0,6.0,8.0,18.0,7,92,43,89.0,0.483146,20.0,99.000000,97.133333,82,55.0,30,0.565811,16.974331,3.025669,3.025669,49.422173,5.577827,5.577827,1993-94
39252,20200814,22002,1610612739,CLE,Cleveland Cavaliers,2003-02-25,CLE vs. NJN,W,240,93,33,77,0.429,3,13.0,0.231,24,27,0.889,14.0,37.0,51.0,17,5.0,7.0,18.0,24.0,3,90,30,64.0,0.468750,11.0,91.122807,101.280702,82,17.0,57,0.186925,10.654717,0.345283,0.345283,15.673122,1.326878,1.326878,2002-03
49088,20600772,22006,1610612741,CHI,Chicago Bulls,2007-02-13,CHI vs. TOR,L,240,111,48,90,0.533,8,13.0,0.615,7,8,0.875,12.0,29.0,41.0,29,6.0,2.0,13.0,22.0,-1,112,40,77.0,0.519481,29.0,99.339623,95.283019,82,49.0,53,0.641056,33.975942,-4.975942,4.975942,47.590610,1.409390,1.409390,2006-07


In [78]:
df[AVG_STATS] = df.sort_values(['SEASON_ID', 'TEAM_ID', 'GAME_DATE']).groupby(['SEASON_ID', 'TEAM_ID'])[STATS].expanding().mean().reset_index((0,1))[STATS]
df[AVG_STATS] = df.sort_values(['SEASON_ID','TEAM_ID', 'GAME_DATE']).groupby(['SEASON_ID', 'TEAM_ID'])[AVG_STATS].shift(1)
df = df.sort_values(['SEASON_ID', 'TEAM_ID', 'GAME_DATE']).set_index('GAME_ID', drop=True)

In [79]:
df.head(10)[['PTS', 'AVG_PTS', 'AST', 'AVG_AST']]

,PTS,AVG_PTS,AST,AVG_AST
GAME_ID,,,,
28500005,91,NaN,25,NaN
28500013,91,91.000000,14,25.000000
28500030,102,91.000000,21,19.500000
28500045,105,94.666667,22,20.000000
28500054,114,97.250000,25,20.500000
28500060,113,100.600000,18,21.400000
28500069,114,102.666667,17,20.833333
28500076,119,104.285714,25,20.285714
28500085,97,106.125000,26,20.875000


In [80]:
df['perc_complete'] = (df['GP'].values-1) / df['TOTAL_GAMES']

In [81]:
home = df.loc[~df['MATCHUP'].str.contains('@')].sort_index()
away = df.loc[df['MATCHUP'].str.contains('@')].sort_index()

In [82]:
home['HOME'] = 1
away['HOME'] = 0

In [83]:
opp_headers = ['OPP_'+stat for stat in STATS+AVG_STATS]

In [84]:
away_opp = away.rename(columns={stat: 'OPP_'+stat for stat in STATS+AVG_STATS})
home_opp = home.rename(columns={stat: 'OPP_'+stat for stat in STATS+AVG_STATS})

In [85]:
data = pd.concat([pd.concat([home, away_opp[opp_headers]], axis=1), pd.concat([away, home_opp[opp_headers]], axis=1)])

In [86]:
data['W'] = data['WL'].map({'W': 1, 'L': 0})

In [87]:
with open('dat/locations.json') as f:
    locations = json.loads(f.read())
data['LOCATION'] = data['TEAM_NAME'].map(locations)
data['LOCATION_W_PCT'] = data['LOCATION'].map(data.loc[data['HOME'] == 1].groupby('LOCATION')['W'].mean().to_dict())

In [88]:
data = data.reset_index()

In [89]:
data.sort_values(['SEASON_ID', 'TEAM_ID', 'GAME_DATE']).head()[['GAME_DATE', 'TEAM_ABBREVIATION', 'MATCHUP', 'PTS', 'AVG_PTS', 'OPP_PTS', 'OPP_AVG_PTS']]

,GAME_DATE,TEAM_ABBREVIATION,MATCHUP,PTS,AVG_PTS,OPP_PTS,OPP_AVG_PTS
21739,1985-10-25,ATL,ATL vs. WAS,91,NaN,100,NaN
58734,1985-10-26,ATL,ATL @ MIL,91,91.000000,117,116.000000
21764,1985-10-29,ATL,ATL vs. NYK,102,91.000000,87,89.000000
58766,1985-11-01,ATL,ATL @ BOS,105,94.666667,109,110.333333
21788,1985-11-02,ATL,ATL vs. PHL,114,97.250000,113,112.500000


*NOTE: OPP_AVG_PTS is not average points scored against, it is AVG_PTS for other team in matchup*

In [90]:
AVG_STATS_A = [stat+'_A' for stat in AVG_STATS]
OPP_STATS = ['OPP_'+stat for stat in STATS]

In [91]:
data[AVG_STATS_A] = data.sort_values(['SEASON_ID', 'TEAM_ID', 'GAME_DATE']).groupby(['SEASON_ID', 'TEAM_ID'])[OPP_STATS].expanding().mean().reset_index((0,1))[OPP_STATS]
data[AVG_STATS_A] = data.sort_values(['SEASON_ID','TEAM_ID', 'GAME_DATE']).groupby(['SEASON_ID', 'TEAM_ID'])[AVG_STATS_A].shift(1)

In [92]:
OPP_AVG_STATS_A = [stat+'_A' for stat in OPP_AVG_STATS]

In [93]:
data = data.set_index('GAME_ID')
home = data.loc[data['HOME']==1].sort_index()
away = data.loc[data['HOME']==0].sort_index()

In [94]:
home[OPP_AVG_STATS_A] = away[AVG_STATS_A]
away[OPP_AVG_STATS_A] = home[AVG_STATS_A]

In [95]:
new = pd.concat([home, away])

In [96]:
phi_2017 = new.loc[(new['SEASON_ID']==22017) & (new['TEAM_ABBREVIATION']=='PHI')].sort_values('GAME_DATE')
phi_2017[['PTS', 'AVG_PTS', 'OPP_PTS', 'AVG_PTS_A']].head()

,PTS,AVG_PTS,OPP_PTS,AVG_PTS_A
GAME_ID,,,,
21700006,115,NaN,120,NaN
21700019,92,115.000000,102,120.000000
21700027,94,103.500000,128,111.000000
21700041,97,100.333333,86,116.666667
21700057,104,99.500000,105,109.000000


In [97]:
phi_2017[['MATCHUP', 'OPP_AVG_PTS', 'OPP_AVG_PTS_A']].head()

,MATCHUP,OPP_AVG_PTS,OPP_AVG_PTS_A
GAME_ID,,,
21700006,PHI @ WAS,NaN,NaN
21700019,PHI vs. BOS,99.5,105.0
21700027,PHI @ TOR,117.0,100.0
21700041,PHI @ DET,108.0,104.0
21700057,PHI vs. HOU,106.0,102.5


In [98]:
new.loc[21700091, ['TEAM_ABBREVIATION', 'MATCHUP', 'AVG_PTS', 'AVG_PTS_A', 'OPP_AVG_PTS', 'OPP_AVG_PTS_A']]

,TEAM_ABBREVIATION,MATCHUP,AVG_PTS,AVG_PTS_A,OPP_AVG_PTS,OPP_AVG_PTS_A
GAME_ID,,,,,,
21700091,GSW,GSW vs. DET,120.0,114.166667,104.5,99.500000
21700091,DET,DET @ GSW,104.5,99.500000,120.0,114.166667


In [99]:
new.to_csv('dat/expanding_avg_1983-2017.csv')